
1. Created a Table name called AuroraBayAlaska in BigQuery using the Console

2. Generated Embeddings using below query in console

```
CREATE OR REPLACE MODEL `AuroraBayAlaska.Embeddings`
REMOTE WITH CONNECTION `us.embedding_conn`
OPTIONS (ENDPOINT = 'text-embedding-005');
```

3. uploaded the dataset from a CSV file,
```
LOAD DATA OVERWRITE AuroraBayAlaska.faq_data
FROM FILES (
    format = 'CSV',
    uris = ['gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv']
);
```
4. generate embeddings using below query
```
CREATE OR REPLACE TABLE `AuroraBayAlaska.faq_data_embedded` AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
    MODEL `AuroraBayAlaska.Embeddings`,
    (SELECT CONCAT(string_field_0, ' ', string_field_1)  AS content  FROM `qwiklabs-gcp-02-8fc93094313c.AuroraBayAlaska.faq_data`)
)
```





In [9]:
from vertexai import init
from vertexai.preview.generative_models import GenerativeModel
import pandas as pd
from google.cloud import bigquery

In [10]:
init(project="qwiklabs-gcp-02-8fc93094313c", location="global")

In [11]:
bq_client = bigquery.Client(project="qwiklabs-gcp-02-8fc93094313c")

In [16]:
def vector_search_data(user_query):
    search_sql = f"""
    CREATE OR REPLACE TABLE `AuroraBayAlaska.faq_result` AS
    SELECT
        query.query,
        base.content
    FROM
        VECTOR_SEARCH(
            TABLE `AuroraBayAlaska.faq_data_embedded`,
            'ml_generate_embedding_result',
            (
                SELECT
                    ml_generate_embedding_result,
                    content AS query
                FROM
                    ML.GENERATE_EMBEDDING(
                        MODEL `AuroraBayAlaska.Embeddings`,
                        (SELECT '{user_query}' AS content)
                    )
            ),
            top_k => 5,
            options => '{{"fraction_lists_to_search": 1.0}}'
        );
    """
    bq_client.query(search_sql).result()

In [17]:
def get_response_from_gemini():
    result_df = bq_client.query("SELECT * FROM `AuroraBayAlaska.faq_result`").to_dataframe()

    context = "\n\n".join(
        result_df.apply(lambda row: f"Q: {row['query']}\nA: {row['content']}", axis=1)
    )

    prompt = (
        "You are an assistant for Aurora Bay. Answer clearly and professionally using only the provided context. If the answer isn’t supported by the context, respond with: “Not able to answer your query.”"
        f"{context}"
    )

    gemini_model = GenerativeModel("gemini-2.5-pro-preview-06-05")
    response = gemini_model.generate_content(prompt)
    return response.text.strip()

In [18]:
user_input = "where is aurora bay located"

vector_search_data(user_input)

response = get_response_from_gemini()

print("Gemini Response:", response)

Gemini Response: Not able to answer your query.


In [15]:
user_input = "How can I apply for a business license in Aurora Bay?"

vector_search_data(user_input)

response = get_response_from_gemini()

print("Response:", response)

Response: Applications for a business license can be submitted either online via the town’s official website or in person at the Town Hall. Processing takes around 2–3 weeks.
